LLMs 
1. Application---> OpenAi API ----> LLM framework
2. Limititons of 1 architecutre- higher cost, no access to internet / real time data, no access to private data sources
3. langchain is a framework that allows you to build applications using LLMs
4. opanAI API keys= 

# Installations
1. pip install openai
2. pip install langchain
3. pip install transformers

In [40]:
# ! pip install openai

In [8]:
# !pip install langchain_community
# !pip install langchain 

In [10]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain #provides all intermediate chains outputs as well .
from langchain_core.output_parsers import StrOutputParser


# TO use open source LLM's locally, we would need to download & isntall OLLama

#from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import pipeline
# pipe = pipeline("text-generation", model="bigscience/bloom")

# Environment Variables

In [11]:
openapi_key= 'xxxxxxxxxxx'
os.environ['OPENAI_API_KEY']= openapi_key

# API Invoke and Basic Prompt Template Creation

In [12]:
llm= OpenAI(temperature= 0.5) # model_name, temp, frequency_penalty,logit_bias,etc are some of the params

In [26]:
name= llm("I want to open a restraunt for italian food. suggest a fancy name for this")
name
parser= StrOutputParser()
#with prompt_template
temp= PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open a restraunt for {cuisine} food. suggest a fancy name for this"
)

name_chain= LLMChain(llm=llm, prompt= temp, output_key='restaurant_name', output_parser= parser)
#name_chain.run({'cusine':'Indian'})

prompt_temp_items= PromptTemplate(
    input_variables= ['restaurant_name'],
    template= 'Provide food menu items for given {restaurant_name}.'
)
food_chain= LLMChain(llm=llm, prompt=prompt_temp_items,output_key= 'menu_items', output_parser= parser)

#Sequential Chain
chain= SequentialChain(
    chains= [name_chain,food_chain],
    input_variables= ['cuisine'],
    output_variables= ['restaurant_name', 'menu_items']
)
res= chain({'cuisine':'Indian'})



{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace"',
 'menu_items': '\n\n1. Butter Chicken\n2. Lamb Biryani\n3. Tandoori Chicken\n4. Palak Paneer\n5. Vegetable Samosas\n6. Chicken Tikka Masala\n7. Naan Bread\n8. Chana Masala\n9. Aloo Gobi\n10. Dal Makhani\n11. Chicken Korma\n12. Mango Lassi\n13. Garlic Naan\n14. Vegetable Biryani\n15. Malai Kofta\n16. Chicken Vindaloo\n17. Paneer Tikka\n18. Chicken Biryani\n19. Aloo Paratha\n20. Gulab Jamun (dessert)'}

Agents will connect with external tools like wikipidea or flights.com or any other tools and then will use LLMs Reasoning capability to execute certain prompts that can't be answered with exiting sept 2021 knowledge

In [40]:
for i, j in  res.items():
    print(i)
    print('\t\t', res[i].strip())
    

cuisine
		 Indian
restaurant_name
		 "Spice Palace"
menu_items
		 1. Butter Chicken
2. Lamb Biryani
3. Tandoori Chicken
4. Palak Paneer
5. Vegetable Samosas
6. Chicken Tikka Masala
7. Naan Bread
8. Chana Masala
9. Aloo Gobi
10. Dal Makhani
11. Chicken Korma
12. Mango Lassi
13. Garlic Naan
14. Vegetable Biryani
15. Malai Kofta
16. Chicken Vindaloo
17. Paneer Tikka
18. Chicken Biryani
19. Aloo Paratha
20. Gulab Jamun (dessert)


# Usecase2 Fintech Docmain--- POC reading tec articles and analysing the fintech data based on differnt prompts

In [4]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredURLLoader

There are many different types of document loader classes available in langchain. eg, csvLoader, TextLoader, UnstructuredURLLoader,etc

In [3]:
#! pip install unstructured libmagic python-magic python-magic-bin # installations for UnstructuredURLLoader class to work.

In [6]:
loader= UnstructuredURLLoader(urls = [
        "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
        "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
    ])

In [7]:
data= loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jatin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [19]:
print(len(data))
print(data[0].metadata['source'])
print(data[0].page_content)


2
https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html
English

Hindi

Gujarati

Specials

Moneycontrol Trending Stock

Infosys INE009A01021, INFY, 500209

State Bank of India INE062A01020, SBIN, 500112

Yes Bank INE528G01027, YESBANK, 532648

Bank Nifty

Nifty 500

Quotes

Mutual Funds

Commodities

Futures & Options

Currency

News

Cryptocurrency

Forum

Notices

Videos

Glossary

All

Hello, Login Hello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardMy AlertsMy MessagesPrice AlertsMy Profile My PROMy PortfolioMy WatchlistFREE Credit Score₹100 Cash RewardMy AlertsMy MessagesPrice AlertsLogoutChat with UsDownload AppFollow us on:

Premium

My Alerts

Elections 2024MarketsHOMEINDIAN INDICESSTOCK ACTIONAll StatsTop GainersTop LosersOnly BuyersOnly Sellers52 Week High52 Week LowPrice ShockersVolume ShockersMost Active StocksGLOBAL MARKETSUS MARKETSBIG SHARK PORTF

After Text loading, the nest step is to split the text into smaller chunks using CharacterTextSplitter or RecurssiveTextSplitter.
The reason behind splitting text is LLM's have token size limits, hence bigger textual data is divided into smaller chunks for processing. (4097 toekns)

In [12]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# splits the text into smaller chunks based on the chunk size and separator we provide.

#usage
text= 'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.\n\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. \nKip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. \nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. \nInterstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.\n\nInterstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014. \nIt has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades.'

splitter_= RecursiveCharacterTextSplitter(
    separators= ["\n\n", "\n", " "],
    chunk_size= 500,
    chunk_overlap= 20,
    length_function= len

    
)

chunks= splitter_.split_text(text)
for chunk in chunks:
    print(len(chunk))

439
380
337
357
253


##### next step is to use save these text chunks into vector databases. eg, of vector databases- pincone, milvus, chroma, FAISS etc.
Here we'll be using FAISS (Facebook AI similarity Search), that is a light weight, in memory vector db type. it allows to quickly search for embeddings of multimedia docs, that are similar to each other. 

## Embeddings Creation and Saving it into FAISS

In [16]:
# !pip install faiss-cpu

In [15]:
# ! pip install sentence-transformers

In [22]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

emb= []
for i in chunks:
    emb.append(get_embedding(i, model='text-embedding-3-small'))
    
dim= len(emb[0])

In [33]:
import faiss
index= faiss.IndexFlatL2(dim) # creating index for faster semantic search
index.add(emb)

AttributeError: 'list' object has no attribute 'shape'

# NAV Emails

In [13]:
import openai
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys


from datetime import datetime
from dotenv import load_dotenv
from langchain import OpenAI, LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
data= pd.read_excel('emails.xlsx')
data.head()

,MailPriority,mail_texts,MailCategoryDesc
0,high,"hi, daily delivery pending for xyz fund which ...",monthly
1,high,"hi chloe, i have not yet had a chance to revi...",expense
2,high,fund category : post approval express max port...,monthly
3,high,fund category : post approval automate max por...,monthly
4,high,"good morning, could we please schedule a wire...",monthly


In [32]:
import openai
from openai import OpenAI

client = OpenAI()

def abstract_summary_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    
    summary= response.choices[0].message.content
    return summary

def key_points_extraction(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    key_points= response.choices[0].message.content
    return key_points

def action_item_extraction(transcription):
    if not isinstance(transcription, str):
        raise ValueError("Transcription must be a string")
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
                },
                {
                    "role": "user",
                    "content": transcription
                }
            ]
        )
        # print(response)
        action_items= response.choices[0].message.content
        return action_items
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def sentiment_analysis(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    sentiment= response.choices[0].message.content
    return sentiment

In [37]:
summary= data['mail_texts'].apply(lambda x:abstract_summary_extraction(x) )
data['summary']= summary
print('summary extracted')

action_items= data['mail_texts'].apply(lambda x:action_item_extraction(x) )
data['action_items']= action_items
print('action items extracted')

sentiment= data['mail_texts'].apply(lambda x:sentiment_analysis(x) )
data['sentiment']= sentiment
print('sentiment extracted')

key_points= data['mail_texts'].apply(lambda x:key_points_extraction(x) )
data['key_points']= key_points
print('key points extracted')


summary extracted
action items extracted
sentiment extracted
key points extracted


In [41]:
data.to_csv('emails_analysis_openai.csv')